<a href="https://colab.research.google.com/github/Habib0905/Vehicle-Pose-Estimation/blob/main/car_pose_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Ultralytics

In [ ]:
!pip install ultralytics

Importing Dependencies

In [2]:
from ultralytics import YOLO
import os
from PIL import Image
import cv2
from google.colab.patches import cv2_imshow

Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Inference with images

In [ ]:
def pad_to_aspect_ratio(image, target_size=(384, 640)):
    width, height = image.size
    aspect_ratio = width / height

    if aspect_ratio > 1:
        new_width = target_size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = target_size[1]
        new_width = int(new_height * aspect_ratio)

    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)
    padded_image = Image.new("RGB", target_size, (128, 128, 128))
    padded_image.paste(resized_image, ((target_size[0] - new_width) // 2, (target_size[1] - new_height) // 2))

    return padded_image

def process_images_in_folder(folder_path, model_path):
    model = YOLO(model_path)
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            padded_image = pad_to_aspect_ratio(image)

            # Save the padded image to Colab session storage
            temp_image_path = '/content/temp_padded_image.jpg'
            padded_image.save(temp_image_path)

            img = cv2.imread(temp_image_path)
            results = model(temp_image_path)[0]

            for result in results:
                bbox = result.boxes.xyxy[0].tolist()

                for kpt in result.keypoints.data[0].tolist():
                    x, y = int(kpt[0]), int(kpt[1])
                    cv2.circle(img, (x, y), radius=3, color=(0, 255, 0), thickness=-1)

            # Display the image with keypoints
            cv2_imshow(img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()

            # Remove the temporary padded image
            os.remove(temp_image_path)

# Folder path for images to be inferred and model path
folder_path = '/content/gdrive/My Drive/car_pose/My Car'
model_path = '/content/gdrive/My Drive/car_pose/runs/pose/train2/weights/last.pt'
process_images_in_folder(folder_path, model_path)


Inference with video

In [ ]:
source =  "/content/gdrive/My Drive/car_pose/My Car/video3.MP4"
model_path = '/content/gdrive/My Drive/car_pose/runs/pose/train2/weights/last.pt'
model = YOLO(model_path)
model.predict(source, save = True, imgsz = 640, vid_stride = 1)

Model Validation (Metrics)

In [ ]:
model_path = '/content/gdrive/My Drive/car_pose/runs/pose/train2/weights/last.pt'
model = YOLO(model_path)  # loading the custom model
metrics = model.val()
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category


In [ ]:
metrics.results_dict  # shows the results of different metrics

In [ ]:
metrics.speed  # speed metrics